In [17]:
import pandas as pd
import numpy as np
import json
import plotly.express as px
import sqlite3

from utils import notebook_util
notebook_util.disp_notebook_full_width()

In [25]:
# from jacob; grabs AA sequence

def get_prot_sequence(pid):
    con=sqlite3.connect("/GeneGraphDB/data/20220322_80kprotein_stats.db")
    cur = con.cursor()
    cmd = "SELECT sequence FROM proteins WHERE pid = '%s'" % pid
    cur.execute(cmd)
    return str(cur.fetchone()[0])
    con.close()

In [26]:
with open("/GeneGraphDB/data/20220308_neo4j_colocalization/cas1.json") as f:
    cas1_dict = json.load(f)

In [27]:
cas1_df = pd.DataFrame(cas1_dict.values())
cas1_df.head()

,tgt_p30,bait_p30,num_tgt_p100s,num_bait_p100s,num_connected_tgt_p100s,num_connected_bait_p100s,num_connections,tgt_colocalization,bait_colocalization,bait_type
0,f232bf64c8a2d86f26,26108e7d6bdf30e691,643,139,9,9,9,0.013997,0.064748,cas1
1,25f95bf2c0720c95c3,26108e7d6bdf30e691,625,139,10,10,10,0.016000,0.071942,cas1
2,691ca9f741d65a3ee9,26108e7d6bdf30e691,422,139,98,137,138,0.232227,0.985612,cas1
3,c0c421cd2905bef4bb,26108e7d6bdf30e691,8,139,4,4,4,0.500000,0.028777,cas1
4,c1eed7f8c50002c763,26108e7d6bdf30e691,2,139,2,2,2,1.000000,0.014388,cas1


In [36]:
cas1_df['weighted_score'] = cas1_df['tgt_colocalization'] * np.power(cas1_df['num_tgt_p100s'], 1/3)

In [34]:
# cas1_df = pd.DataFrame(cas1_dict.values())
print(len(cas1_df), "rows total")
cas1_df = cas1_df[cas1_df['num_tgt_p100s'] >= 5]
print(len(cas1_df), "filtered for >5 tgt p100")
cas1_df = cas1_df[cas1_df['num_connections'] > 1]
print(len(cas1_df), "+ filtered for >1 shared edges")

cas1_df = cas1_df[cas1_df['tgt_colocalization'] > 0.1]
print(len(cas1_df), "+ filtered for >0.1 tgt colocalization")

cas1_df = cas1_df[cas1_df['bait_colocalization'] > 0.01]
print(len(cas1_df), "+ filtered for >0.01 bait colocalization")


cas1_df.sort_values('weighted_score', ascending=False, inplace=True)
cas1_df
# cas1_df.sort_values('tgt_colocalization', ascending=False)

12521 rows total
12521 filtered for >5 tgt p100
12521 + filtered for >1 shared edges
12521 + filtered for >0.1 tgt colocalization
12521 + filtered for >0.01 bait colocalization


,tgt_p30,bait_p30,num_tgt_p100s,num_bait_p100s,num_connected_tgt_p100s,num_connected_bait_p100s,num_connections,tgt_colocalization,bait_colocalization,bait_type,weighted_score
106,2df6766ad7d9fe5716,37407b2b7828caf2f8,286,1033,274,289,291,0.958042,0.279768,cas1,2.969325
70588,9bac3f53f9f92ff1db,557e309d5f3d13e440,445,781,385,171,394,0.865169,0.218950,cas1,2.929360
70590,3c157c75f39f8ab820,557e309d5f3d13e440,689,781,544,238,557,0.789550,0.304738,cas1,2.917585
76186,40f22d7658756e33a0,f0f5ad3fcba8e5170b,325,1261,290,50,314,0.892308,0.039651,cas1,2.837209
76153,d55162057d1113e638,f0f5ad3fcba8e5170b,484,1261,397,145,547,0.820248,0.114988,cas1,2.824322
...,...,...,...,...,...,...,...,...,...,...,...
67458,0559da88a58362d4b1,2b9b82deec52e5bce5,9,13,1,2,2,0.111111,0.153846,cas1,0.172427
29533,656e8c5fd55445b9c3,03cb4a497fda47d1a9,9,58,1,2,2,0.111111,0.034483,cas1,0.172427
17345,8196c771ead47e9100,94505389b83799096c,9,12,1,2,2,0.111111,0.166667,cas1,0.172427
8630,2f00e466e45b9cc176,dbd09fbfc7a4fffaf4,9,150,1,2,2,0.111111,0.013333,cas1,0.172427


In [35]:
for tgt_p30 in cas1_df['tgt_p30'][0:10]:
    print(tgt_p30)
    print()
    print(get_prot_sequence(tgt_p30))
    display(cas1_df[(cas1_df['tgt_p30'] == tgt_p30) | (cas1_df['bait_p30'] == tgt_p30)])
    print("\n\n*********\n")

2df6766ad7d9fe5716

MAWGTNTESGFDFQTYGENRREPVDFDGLRLVLFKPLEDDE*


,tgt_p30,bait_p30,num_tgt_p100s,num_bait_p100s,num_connected_tgt_p100s,num_connected_bait_p100s,num_connections,tgt_colocalization,bait_colocalization,bait_type,weighted_score
106,2df6766ad7d9fe5716,37407b2b7828caf2f8,286,1033,274,289,291,0.958042,0.279768,cas1,2.969325




*********

9bac3f53f9f92ff1db

MKKTRAFLKWAGGKYPLVEDIKKHLPQGDCLIEPFVGAGSVFLNTSYDAYVLADINSDLINLYNTVKYRTDSFIGEVREIFTPEFNTAEQYY*


,tgt_p30,bait_p30,num_tgt_p100s,num_bait_p100s,num_connected_tgt_p100s,num_connected_bait_p100s,num_connections,tgt_colocalization,bait_colocalization,bait_type,weighted_score
70588,9bac3f53f9f92ff1db,557e309d5f3d13e440,445,781,385,171,394,0.865169,0.21895,cas1,2.92936




*********

3c157c75f39f8ab820

MRSPRRDAGINHIGELGAIMERVTVTLGERSYPITIAAGLFNDPASFMPLKAGDQVMLVTNQTLAPLYLEQVRNVLEQSGVVVDQVILPDGEQYKSLAVLEQVFSALLAKPHGRDTTLVALGGGVIGDLTGFAAACLPARRTFYSGADHAAVAGGLLVGGKNRRQSSAR*


...



*********

c47586037217d8ea14

MLVIVLENAPPRLRGRLAIWLLEIRAGVYAGNYSAKVRDHIWSHANRESKTAMR*


,tgt_p30,bait_p30,num_tgt_p100s,num_bait_p100s,num_connected_tgt_p100s,num_connected_bait_p100s,num_connections,tgt_colocalization,bait_colocalization,bait_type,weighted_score
117,c47586037217d8ea14,37407b2b7828caf2f8,176,1033,170,172,173,0.965909,0.166505,cas1,2.716682




*********

4711420acbe5fce9b8

MRSLKPITIFGESILPGENKTINVEMARLHTTTKLNIPIIVRRSKQEGPVILFTAGIHGDEINGIEIVRQLISQKINRPKRGTIICIPIINMYGFVNRSREFPDGRDLNRSFPGSKKGSLASRFAFHILNEVMPEVHYAIDFHAGGASRFNAPQIRLAENDAELKQLADVFNAPFTLYSKNIAGSFRNSSTKMNVKMLLFEGGKSLDIDKKVAREGLEGTKRVLAQLDMLDPKHNIEVPENKTIYIEKSNWVRAKCSGMLHDLNTIGTFVTKGTVLALITDPFGKFERKVKAPNDGYVINSNQSPIVYEGDAIFHLSTSLVGTKSNRIEKPEAYIQQFNDIEIFKT*


,tgt_p30,bait_p30,num_tgt_p100s,num_bait_p100s,num_connected_tgt_p100s,num_connected_bait_p100s,num_connections,tgt_colocalization,bait_colocalization,bait_type,weighted_score
64631,4711420acbe5fce9b8,aef2ee771335d7b3a9,161,3350,157,159,159,0.975155,0.047463,cas1,2.694257




*********



'MKKYKLGLDVGVSSVGVAVIKEENGKSSVEKLSVRIVPEDPNFHGKFYSGNTASKNLDRTIKRGIRKNNQRFKARRDKLYQTLRENNMFPTDNLFDLPATELYGLRAKAVSKKVSLEELGRIFIVLNQRRGFLSNRKSVSEEENSTEYKECIATLEKELAGRTIGQKLYSELEESESDFEVLLRERTYQRSSYIEEFDRIWDEQKKYYSVLTGGANEDNNKETLYDLIRNRIIYYQRPLKSQKGLISECSFEKYHKATAKSSPYFELFRIWQKVNDLSWKTPGGETFKPTLGQKQNLKEALWNGENLNAKHKLSVTEIKKLLGYGRNEKIYLNFTELDGSRTYTVLKNALEEAGIENFQQYLFFNLDISDEKGGLFELWHITYSLPTEKEVVNALKNRFGFSENQSEIIAKKVGYPSNYGNLSTRAIRKLLPYLEKGLGYSEACDAVGYDHSGYKTEIELKPNLKQLKKNELRNPVVEQILNQVVNVVNLVIEKYGKIDEIRVELARELRNNAKTRKNISQANSKNKRNNDVIRKKLIEEYGFRLVNGRDVKRYILWEETAHECLYCNNPINGTDMLTGNADIEHVLPKSRSFSNAMNNYIIAHRKCNSNKGQKTAYDFMESKGEEVLGQYIEKVNALYNDGKGSISKSKFDNLLCKGEDIPSDFVERMKKDSQYISKEVVKMLKTVCPNVYTTTGQVTDFLREEWKLKNLLQELTFDKYKAIGQVEEKTIKTKNGQTKTFETITDWSKRDDHRHHAVDALICALTNQKIIFKLNNLNKIYQYERDALSQKEIEEFEEFLEGKFSLKEFSTQQENIIECSIPNIRREVQKHLEEIFISFKKENSKVLTKNINAPKNVEPKTTWVPRARLHEDTIMGRVKRIAEKKMKLSDKVSMEEIEMVVNPEIKEILYQHLTKFDNNPKIAFAQKTLKGELISYKGNEIKEVGVYEWVNTKRVNVENFLTDNQKTTSAKDKFIENILKSDRGIGVLLDRRLKEFNGD

In [ ]:
fig = px.histogram(cas1_df[cas1_df["num_tgt_p100s"] >= 5], x="num_tgt_p100s")
fig.show()

In [ ]:
cas1_df = cas1_df[cas1_df["num_tgt_p100s"] >= 5]